In [ ]:
#Train test split
import pandas as pd
from sklearn.model_selection import train_test_split

# Assume df is your original DataFrame with 'target' as the label column

# Split the dataset into two subsets based on the target label
df_class_0 = df[df['target'] == 0]
df_class_1 = df[df['target'] == 1]

# Perform EDA separately on df_class_0 and df_class_1 here

# After EDA, merge the subsets back together, with class 0 data first
df_merged = pd.concat([df_class_0, df_class_1])

# Extract features and labels
X = df_merged.drop(columns=['target'])
y = df_merged['target']

# Perform a stratified train-test split to maintain the class distribution
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Check the class distribution in the train and test sets
print(f"Training set class distribution: {y_train.value_counts(normalize=True)}")
print(f"Test set class distribution: {y_test.value_counts(normalize=True)}")


## Some links of interests

### MFEUsLNet
https://www.sciencedirect.com/science/article/pii/S2215098624000181#b0155  

From the state of the art:

Ghosh et al. [31] proposed **SkinNet-16**, a deep-learning model aimed at distinguishing between benign and malignant skin lesions. Their approach employs advanced neural network architecture to enhance classification accuracy.

*Pronab Ghosh, Sami Azam, Ryana Quadir, Asif Karim, F.M. Shamrat, Shohag Kumar Bhowmik, Mirjam Jonkman, Khan Md Hasib, Kawsar Ahmed, SkinNet-16: a deep learning approach to identify benign and malignant skin lesions, Front. Oncol. 12 (2022) 931141.*

### EfficientNet V2 B0
https://www.kaggle.com/code/matthewjansen/transfer-learning-skin-cancer-classification  

### Inception-RestNet-v2
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9759648/


### Sample Code for using Optuna

In [ ]:
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

# Load example dataset
data = load_iris()
X, y = data.data, data.target
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 10, 200)
    max_depth = trial.suggest_int('max_depth', 2, 32)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)
    max_features = trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2', None])
    
    # Create the model with these hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        random_state=42
    )
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Predict on the validation set
    preds = model.predict(X_val)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_val, preds)
    
    # We want to maximize accuracy, so return it as a negative value
    return accuracy

# Create a study object and specify the direction of optimization
study = optuna.create_study(direction='maximize')

# Start the optimization
study.optimize(objective, n_trials=100, n_jobs=-1)  # n_trials is the number of trials, n_jobs=-1 uses all CPUs

# Get the best trial
best_trial = study.best_trial

print(f'Best trial number: {best_trial.number}')
print(f'Best value (accuracy): {best_trial.value}')
print(f'Best hyperparameters: {best_trial.params}')

# XGBoost imbalance classification

https://machinelearningmastery.com/xgboost-for-imbalanced-classification/